In [4]:
from flask import Flask, request, jsonify
from pymongo import MongoClient

# Initialize Flask app
app = Flask(__name__)

# Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')
db = client['twitter_db']
tweets_collection = db['tweets']

# Query: Tweets per day
@app.route('/tweets_per_day', methods=['GET'])
def tweets_per_day():
    term = request.args.get('term', '')  # Get the search term from the request
    if term:
        results = list(tweets_collection.aggregate([
            {"$match": {"text": {"$regex": term, "$options": "i"}}},
            {"$group": {"_id": {"$substr": ["$created_at", 0, 10]}, "count": {"$sum": 1}}}
        ]))
        return jsonify(results)
    return jsonify({"error": "Please provide a valid search term"}), 400

# Query: Unique users posting the term
@app.route('/unique_users', methods=['GET'])
def unique_users():
    term = request.args.get('term', '')  # Get the search term from the request
    if term:
        users_count = len(tweets_collection.distinct("user_id_str", {"text": {"$regex": term, "$options": "i"}}))
        return jsonify({"unique_users": users_count})
    return jsonify({"error": "Please provide a valid search term"}), 400

# Query: Average likes for tweets containing the term
@app.route('/avg_likes', methods=['GET'])
def avg_likes():
    term = request.args.get('term', '')  # Get the search term from the request
    if term:
        result = list(tweets_collection.aggregate([
            {"$match": {"text": {"$regex": term, "$options": "i"}}},
            {"$group": {"_id": None, "avg_likes": {"$avg": "$favorite_count"}}}
        ]))
        if result:
            return jsonify(result[0])
        return jsonify({"avg_likes": 0})
    return jsonify({"error": "Please provide a valid search term"}), 400

# Home route to display available endpoints
@app.route('/')
def home():
    return """
    <h1>Twitter Data Analytics API</h1>
    <p>Use the following endpoints:</p>
    <ul>
        <li>/tweets_per_day?term=YOUR_TERM</li>
        <li>/unique_users?term=YOUR_TERM</li>
        <li>/avg_likes?term=YOUR_TERM</li>
    </ul>
    """

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (windowsapi)


SystemExit: 1